In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# -----------------------------
# CVaR utility
# -----------------------------
def cvar_loss(losses: torch.Tensor, alpha: float) -> torch.Tensor:
    sorted_losses, _ = torch.sort(losses, descending=True)
    k = max(1, int((1 - alpha) * len(sorted_losses)))
    return sorted_losses[:k].mean()

# -----------------------------
# DCGAN models (64×64)
# -----------------------------
class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512), nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256), nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        return self.net(z.view(z.size(0), z.size(1), 1, 1))

class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False), nn.BatchNorm2d(512), nn.LeakyReLU(0.2, True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False)
        )

    def forward(self, x):
        return self.net(x).view(-1)

# -----------------------------
# Training step
# -----------------------------
def train_gan(G, D, loader, optG, optD, device, alpha=None):
    relu = nn.ReLU()
    G.train(); D.train()
    d_losses, g_losses = [], []
    for real, _ in loader:
        real = real.to(device)
        bs = real.size(0)

        # Discriminator update
        optD.zero_grad()
        real_scores = D(real)
        z = torch.randn(bs, G.net[0].in_channels, device=device)
        fake = G(z).detach()
        fake_scores = D(fake)
        d_loss = (relu(1 - real_scores) + relu(1 + fake_scores)).mean()
        d_loss.backward()
        optD.step()

        # Generator update
        optG.zero_grad()
        z = torch.randn(bs, G.net[0].in_channels, device=device)
        gen = G(z)
        gen_scores = D(gen)
        gen_losses = -gen_scores
        g_loss = gen_losses.mean() if alpha is None else cvar_loss(gen_losses, alpha)
        g_loss.backward()
        optG.step()

        d_losses.append(d_loss.item())
        g_losses.append(g_loss.item())

    return sum(d_losses) / len(d_losses), sum(g_losses) / len(g_losses)

# -----------------------------
# Main comparison
# -----------------------------
def main():
    # Hyperparameters
    epochs = 25
    batch_size = 128
    lr = 2e-4
    alpha = 0.95  # CVaR level

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data loader
    transform = transforms.Compose([
        transforms.CenterCrop(178),
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5,)*3, (0.5,)*3),
    ])
    celeba = datasets.CelebA("/Users/jingfutan/Documents/GitHub/DSCI-498-Course-Project/celeba_data", split="train", download=True, transform=transform)
    loader = DataLoader(celeba, batch_size=batch_size, shuffle=True, num_workers=4)

    # Models
    G_std, D_std = Generator().to(device), Discriminator().to(device)
    G_cvar, D_cvar = Generator().to(device), Discriminator().to(device)

    # Optimizers
    optG_std = optim.Adam(G_std.parameters(), lr=lr, betas=(0.5,0.999))
    optD_std = optim.Adam(D_std.parameters(), lr=lr, betas=(0.5,0.999))
    optG_cvar = optim.Adam(G_cvar.parameters(), lr=lr, betas=(0.5,0.999))
    optD_cvar = optim.Adam(D_cvar.parameters(), lr=lr, betas=(0.5,0.999))

    # Training
    losses_std, losses_cvar = {"d":[], "g":[]}, {"d":[], "g":[]}
    for ep in range(1, epochs+1):
        ds, gs = train_gan(G_std, D_std, loader, optG_std, optD_std, device, alpha=None)
        dc, gc = train_gan(G_cvar, D_cvar, loader, optG_cvar, optD_cvar, device, alpha=alpha)
        losses_std["d"].append(ds); losses_std["g"].append(gs)
        losses_cvar["d"].append(dc); losses_cvar["g"].append(gc)
        print(f"Epoch {ep:2d} | Std D={ds:.3f}, G={gs:.3f} | CVaR D={dc:.3f}, G={gc:.3f}")

    # Plot losses
    plt.figure(figsize=(10,4))
    plt.plot(losses_std["g"], label="Gen Std")
    plt.plot(losses_cvar["g"], label="Gen CVaR")
    plt.plot(losses_std["d"], label="Disc Std")
    plt.plot(losses_cvar["d"], label="Disc CVaR")
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.title("GAN vs CVaR-GAN Training Losses")
    plt.show()

    # Sample visualization
    G_std.eval(); G_cvar.eval()
    with torch.no_grad():
        z = torch.randn(16, 100, device=device)
        samp_std = G_std(z).cpu()
        samp_cvar = G_cvar(z).cpu()

    fig, axs = plt.subplots(2, 8, figsize=(12,3))
    for i in range(8):
        img_std = (samp_std[i].permute(1,2,0)*0.5 + 0.5).clamp(0,1)
        img_cvar = (samp_cvar[i].permute(1,2,0)*0.5 + 0.5).clamp(0,1)
        axs[0,i].imshow(img_std); axs[0,i].axis("off")
        axs[1,i].imshow(img_cvar); axs[1,i].axis("off")
    axs[0,0].set_ylabel("Std GAN", rotation=0, labelpad=30)
    axs[1,0].set_ylabel("CVaR GAN", rotation=0, labelpad=30)
    plt.tight_layout(); plt.show()

if __name__ == "__main__":
    main()


Files already downloaded and verified


libc++abi: libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x123156ac0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1568, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/anaconda3/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/multip

KeyboardInterrupt: 